Load required packages and data files for further analysis.

In [22]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import math
import random

Defined Functions Needed For the Following Analysis

In [2]:
#### T-transformation
def t(mtx):
    mtx_t = []
    for j in range( 0, len(mtx[0]) ):
        mtx_t.append( [mtx[i][j] for i in range(0,len(mtx))] )
    return(mtx_t)

# conversion list of str to list of int
def convert_to_float(list_str):
    list_float = [float(i) for i in list_str]
    return(list_float)

# source code from https://www.geeksforgeeks.org/python-get-unique-values-list/
# function to get unique values 
def unique(input_list): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in input_list: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x)
    return(unique_list)

## RowSums & RowAverage Function
def rowSums(mtx):
    res = list()
    for j in range(0, len(mtx[0])): 
        tmp = 0
        for i in range(0, len(mtx)): 
            tmp = tmp + mtx[i][j]
        res.append(tmp)
    return(res)

def rowAverage(mtx):
    res = list()
    for j in range(0, len(mtx[0])): 
        tmp = 0
        for i in range(0, len(mtx)): 
            tmp = tmp + mtx[i][j]
        res.append(tmp/len(mtx))
    return(res)

In [9]:
DATA_FOLDER="/Users/kangb3/github/project_spring_2020_dup/"
EXP_FNAME = os.path.join(DATA_FOLDER, "exprs_mtx.txt")
META_FNAME = os.path.join(DATA_FOLDER, "metatable.txt")

In [10]:
#### Read Meta Files - read as list of lists
meta = []
with open(META_FNAME) as f_obj:
    for line in f_obj:
        res = line.replace('"','').strip('\n').split('\t')
        meta.append(res)

In [11]:
### Insert one element in the 1th list of the meta data
meta[0].insert(0,'rowname')

In [12]:
### Read matrix file - read as list of lists
mtx = []
with open(EXP_FNAME) as f_obj:
    for line in f_obj:
        res = line.replace('"','').strip('\n').split('\t')
        mtx.append(res)

In [13]:
### Insert one element in the 1th list of the mtx
mtx[0].insert(0,'gene')

In [14]:
# Apply T-transformation and convert the list of lists to Dictionary with cell.id as keys and gene expression as value
mtx_t = t(mtx)
mtx_dic = {}
for line in mtx_t:
    mtx_dic[line[0]] = line[1:]

Index Generator

In [15]:
temp = [meta[i][2] for i in range(0,len(meta))] # extract cell cluster info
unique_cluster = unique(temp[1:])               # take unique cluster info
unique_cluster = sorted(unique_cluster)
meta_by_cluster = []
meta_dic = {}

## group cell.ids based on their cluster
for cluster in unique_cluster:
    index = [i for i, n in enumerate(temp) if n == cluster] # extract the indices of rows in a given cluster
    a = [meta[i][1:3] for i in index] # using the indices, extract cell.id and assigned cluster info.
    meta_by_cluster.append(a)

# convert the list acquired above to dictionary with the cluster as keys
for j in range(0, len(unique_cluster)):
    meta_dic[ meta_by_cluster[j][0][1] ] = [meta_by_cluster[j][i][0] for i in range(0, len(meta_by_cluster[j]))]

Random Selector

In [16]:
## Random selection of cell.ids from each cluster
## And make dic

dic_random_items = {}
num_to_select = 18    # this will be determined by user.

for key in meta_dic.keys():
    group_of_items = meta_dic[key].copy()
    
    if len(meta_dic[key]) < num_to_select:
        print("Number of items to select exceeds the number of items in the list.")
        print("Remember this is random selection without replacement.")
        cycle_to_run = math.floor(len(meta_dic[key])/num_to_select)
    elif len(meta_dic[key])%num_to_select == 0:
        cycle_to_run = int(len(meta_dic[key])/num_to_select)
    else:
        cycle_to_run = math.ceil(len(meta_dic[key])/num_to_select)
    
    for i in range(0, cycle_to_run): # random sampling from the given list without replacement
        if len(group_of_items) > num_to_select:
            a = random.sample(group_of_items, num_to_select)
            dic_random_items[ a[0] ] = a
            [group_of_items.remove(a[i]) for i in range(0,len(a))] # selected cell.ids are removed from the dic.
        elif len(group_of_items) < num_to_select: # the remaining cells are taken 
            a = group_of_items
            dic_random_items[ a[0] ] = a

Averaging Selected Rows

In [17]:
mtx_small = {}
a = mtx_dic['gene'].copy()
mtx_small['gene'] = a
for key, values in dic_random_items.items():
    temp = []
    a = []
    for value in values:
        a = mtx_dic[ value ].copy()
        temp.append(convert_to_float(a))
    mtx_small[ key ] = rowAverage(temp)

Dictionary to List of Lists (Matrix)

In [18]:
[v.insert(0,k) for k, v in mtx_small.items()]
temp_2 = [x for x in mtx_small.values()]

In [19]:
final_mtx = []
temp = t(temp_2)
final_mtx = temp.copy()

Export the result as TXT file

In [19]:
## Export result matrix as txt file
with open("mtx_small.txt", "w") as file:
    for lines in final_mtx:
        for line in lines:
            file.write(str(line) + "\t")
        file.write("\n")

In [114]:
## Export the list of randomly selected cells and representative cells.
import csv

with open('randomcells.csv', 'w') as f:
    for key in dic_random_items.keys():
        f.write("%s, %s\n" % (key, dic_random_items[key]))

In [53]:
meta_by_clus_temp = []
for i in range(0, len(meta_by_cluster)):
    for j in range(0, len(meta_by_cluster[i])):
        meta_by_clus_temp.append(meta_by_cluster[i][j])

meta_by_clus_dic = {}
for i in range(0, len(meta_by_clus_temp)):
    meta_by_clus_dic[ meta_by_clus_temp[i][0] ] = meta_by_clus_temp[i][1]

In [54]:
meta_by_clus_dic

{'TTGGTATTTTGA': '0',
 'TGTCAGCATTCC': '0',
 'AGGTAATTTAAT': '0',
 'ACGTTACGGAGT': '0',
 'TGTTCGTCGGGC': '0',
 'TAGGGGGCTTTT': '0',
 'AGAATCATCGAG': '0',
 'TCGGCCGAAGGC': '0',
 'AATAACCTGTAG': '0',
 'CCTTAAAGCCCA': '0',
 'ATTGGTTGATTT': '0',
 'TGGCCTGAATCG': '0',
 'CATATAGCTGGG': '0',
 'GCCTACCTCCCN': '0',
 'AGTTCTCCATGC': '0',
 'GCCACAGATCAA': '0',
 'AGAATGAGCTAT': '0',
 'TGAAGCCAGGAA': '0',
 'GCCCATCATTTN': '0',
 'GTTTGACAGGCN': '0',
 'ATTGTATAGACN': '0',
 'GCTCCGCTTACC': '0',
 'CGGCATTGGCAG': '0',
 'AGTCACCATCGG': '0',
 'CCTTTCCATTAG': '0',
 'CGACTTCAAGTG': '0',
 'ATAGTTCTTCAG': '0',
 'TCTCAGTGAGTG': '0',
 'CAAACTGGGGCC': '0',
 'AGTTTCACTAAN': '0',
 'CCTTTCAAGAAN': '0',
 'CATGATCCTTGC': '0',
 'GAGTTAAAGGTN': '0',
 'TGGACGCTTTCG': '0',
 'CTCGGTTAAGTG': '0',
 'TTAACTGGAGCN': '0',
 'ATAGTAGGGATG': '0',
 'TGGGAGTCCTTG': '0',
 'GTTCACCCGAAC': '0',
 'GCGCATTCAGAG': '0',
 'TGGTCTAATCTG': '0',
 'TCAACGTCGGCA': '0',
 'TGGTTTTTCGGG': '0',
 'ACGGATAGAAAN': '0',
 'GGCGCTCCTATN': '0',
 'GATCAGTT

In [55]:
for k, v in dic_random_items.items():
    print( "Represeting Cell ID: " + k + " - Assigned Cluster: " + str(meta_by_clus_dic[k]) + " - Sample Number: " + str(len(v)) )

Represeting Cell ID: CCAGATGACGCT - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: TTAACTGGAGCN - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: AATAGCTTGCAT - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: CTAGTTTGTGCN - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: AAGAACCGCCGN - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: GCCCATCATTTN - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: CTCGGTTAAGTG - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: GCAACGTCCAGN - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: TGGACGCTTTCG - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: GTCCTACGAATC - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: CGGTCCCAGGGT - Assigned Cluster: 0 - Sample Number: 18
Represeting Cell ID: TGTTCGTCGGGC - Assigned Cluster: 0 - Sample Number: 2
Represeting Cell ID: CGGTGGAAGGCT - Assigned Cluster: 1 - Sample Number: 18
Represeting C